In [33]:
import numpy as np

In [34]:
class TicTacToe:
    def __init__(self, size=3):
        
        self.size = size
        self.board = np.zeros((self.size, self.size))
        self.current_turn = 1
    
    
    def make_move(self, row, col):
        
        if self.board[row, col] == 0:
            
            self.board[row, col] = self.current_turn
            self.current_turn = -self.current_turn
            
            return True
        
        return False
    

    def get_available_moves(self):
        
        return [(row, col) for row in range(self.size) for col in range(self.size) if self.board[row, col] == 0]


    def check_winner(self):

        for i in range(self.size):

            if abs(self.board[i, :].sum()) == self.size: 
                return self.board[i, 0]
            
            if abs(self.board[:, i].sum()) == self.size: 
                return self.board[0, i]
            
        if abs(np.diag(self.board).sum()) == self.size: 
            return self.board[0, 0]
        
        if abs(np.diag(np.fliplr(self.board)).sum()) == self.size: 
            return self.board[0, -1]
        
        if np.all(self.board != 0):
            return 0 
        
        return None


In [36]:
def play_game():
        
    game = TicTacToe()
    

    while game.check_winner() is None:
        print(f"Turn: {'player 1' if game.current_turn == 1 else 'player 2'}")
        row_move = int(input('Row move: '))
        col_move = int(input('Column move: '))

        game.make_move(row_move, col_move)

        
        print(game.board)
        print("----------")

        winner = game.check_winner()
        if winner is not None:
            print("Winner:", 'player 1' if winner == 1 else 'player 2' if winner == -1 else 'Draw')
            break

play_game()

Turn: player 1
[[1. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
----------
Turn: player 2
[[ 1. -1.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]]
----------
Turn: player 1
[[ 1. -1.  0.]
 [ 1.  0.  0.]
 [ 0.  0.  0.]]
----------
Turn: player 2
[[ 1. -1.  0.]
 [ 1.  0.  0.]
 [-1.  0.  0.]]
----------
Turn: player 1
[[ 1. -1.  1.]
 [ 1.  0.  0.]
 [-1.  0.  0.]]
----------
Turn: player 2
[[ 1. -1.  1.]
 [ 1. -1.  0.]
 [-1.  0.  0.]]
----------
Turn: player 1
[[ 1. -1.  1.]
 [ 1. -1.  0.]
 [-1.  1.  0.]]
----------
Turn: player 2
[[ 1. -1.  1.]
 [ 1. -1.  0.]
 [-1.  1. -1.]]
----------
Turn: player 1
[[ 1. -1.  1.]
 [ 1. -1.  1.]
 [-1.  1. -1.]]
----------
Winner: Draw


In [35]:
class MonteCarloSearch:
    def __init__(self):
        pass

    def traverse():
        pass

    def rollout():
        pass

    def rollout_policy():
        pass

    def backpropagate():
        pass

    def best_child():
        pass

In [37]:

game = TicTacToe(size=3)

game.board[0,0] = -1
game.board[1,1] = 1 

print(game.board)
    

[[-1.  0.  0.]
 [ 0.  1.  0.]
 [ 0.  0.  0.]]


In [32]:
import numpy as np
import random
import math
import tkinter as tk

class TicTacToe:
    def __init__(self, size=3):
        self.board = np.zeros((size, size), dtype=int)
        self.current_turn = 1  # 1 for 'X', -1 for 'O'

    def make_move(self, row, col):
        if self.board[row, col] == 0:
            self.board[row, col] = self.current_turn
            self.current_turn = -self.current_turn  # Switch turns
            return True
        return False

    def check_winner(self):
        for i in range(3):
            if abs(self.board[i, :].sum()) == 3: return self.board[i, 0]
            if abs(self.board[:, i].sum()) == 3: return self.board[0, i]
        if abs(np.diag(self.board).sum()) == 3: return self.board[0, 0]
        if abs(np.diag(np.fliplr(self.board)).sum()) == 3: return self.board[0, -1]
        if np.all(self.board != 0):
            return 0  # Draw
        return None

    def get_available_moves(self):
        return [(i, j) for i in range(3) for j in range(3) if self.board[i, j] == 0]

    def copy(self):
        new_game = TicTacToe(size=self.board.shape[0])
        new_game.board = np.copy(self.board)
        new_game.current_turn = self.current_turn
        return new_game

class MCTSNode:
    def __init__(self, game, parent=None, move=None):
        self.game = game
        self.parent = parent
        self.move = move
        self.wins = 0
        self.visits = 0
        self.children = []
        self.untried_moves = game.get_available_moves()

    def UCB1(self):
        return self.wins / self.visits + math.sqrt(2) * math.sqrt(math.log(self.parent.visits) / self.visits) if self.parent else float('inf')

    def select_child(self):
        return max(self.children, key=lambda c: c.UCB1())

    def add_child(self, move):
        new_game = self.game.copy()
        new_game.make_move(*move)
        child_node = MCTSNode(new_game, parent=self, move=move)
        self.untried_moves.remove(move)
        self.children.append(child_node)
        return child_node

    def update(self, result):
        self.visits += 1
        if self.parent is not None:
            player = self.parent.game.current_turn
            if result == player:
                self.wins += 1
            elif result == 0:  # Draw
                self.wins += 0.5
        else:
            if (self.game.current_turn == -1 and result == 1) or (self.game.current_turn == 1 and result == -1):
                self.wins += 1
            elif result == 0:
                self.wins += 0.5

def MCTS(root, iterations=10000):
    for _ in range(iterations):
        node = root
        game = root.game.copy()

        while node.untried_moves == [] and node.children != []:
            node = node.select_child()
            game.make_move(*node.move)

        if node.untried_moves != []:
            move = random.choice(node.untried_moves)
            game.make_move(*move)
            node = node.add_child(move)

        while game.get_available_moves() != []:
            move = random.choice(game.get_available_moves())
            game.make_move(*move)

        while node is not None:
            node.update(game.check_winner())
            node = node.parent


class TicTacToeGUI:
    def __init__(self, master):
        self.master = master
        self.game = TicTacToe()
        self.buttons = [[None for _ in range(3)] for _ in range(3)]
        self.initialize_board()

    def initialize_board(self):
        for i in range(3):
            for j in range(3):
                button = tk.Button(self.master, text='', font=('Arial', 24), height=2, width=5,
                                   command=lambda row=i, col=j: self.on_click(row, col))
                button.grid(row=i, column=j)
                self.buttons[i][j] = button

    def on_click(self, row, col):
        if self.game.make_move(row, col):
            self.update_buttons()
            winner = self.game.check_winner()
            if winner is not None:
                self.end_game(winner)
            else:
                self.ai_move()

    def update_buttons(self):
        for i in range(3):
            for j in range(3):
                text = {1: 'X', -1: 'O', 0: ''}[self.game.board[i, j]]
                self.buttons[i][j].config(text=text)

    def ai_move(self):
        root = MCTSNode(self.game)
        MCTS(root, iterations=10000)
        move = root.select_child().move
        self.game.make_move(*move)
        self.update_buttons()
        winner = self.game.check_winner()
        if winner is not None:
            self.end_game(winner)

    def end_game(self, winner):
        result = {1: "X wins!", -1: "O wins!", 0: "Draw!"}[winner]
        message = tk.Message(self.master, text=result, width=200)
        message.grid(row=3, column=0, columnspan=3)
        for i in range(3):
            for j in range(3):
                self.buttons[i][j].config(state='disabled')

# TicTacToe, MCTSNode, and MCTS functions/classes remain unchanged from previous examples.

def main():
    root = tk.Tk()
    root.title("Tic-Tac-Toe AI")
    app = TicTacToeGUI(root)
    root.mainloop()

if __name__ == "__main__":
    main()
